In [8]:
import pandas as pd
import psycopg2
import numpy as np
import sklearn
import pyodbc

In [17]:
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random)
filepath = '1mioraw.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 200                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_sql_query("""Select content, type_id
from article""", connection)

In [45]:
# VIGTIG NOTE: måden jeg opdeler sættet på er MEGET dårlig, på det endelige skal vi shuffle og derefter opdele! 

In [ ]:
### Nearest hood (neighbors)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

In [47]:
from sklearn.neighbors import NearestCentroid

X = np.array(x.toarray()[int(len(df["type_id"])/2):])
y = np.array(df["type_id"][int(len(df["type_id"])/2):])
clf = NearestCentroid()
clf.fit(X, y)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [48]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(test_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 63


In [29]:
### Naive bayes

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 125 points : 57


In [42]:
### Decision trees

In [61]:
from sklearn import tree

X = x.toarray()[int(len(df["type_id"])/2):]
y = df["type_id"][int(len(df["type_id"])/2):]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_X, train_y)

In [62]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(train_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 0


In [63]:
### Support Vector Machines

In [67]:
from sklearn import svm
X = x.toarray()[int(len(df["type_id"])/2):]
y = df["type_id"][int(len(df["type_id"])/2):]
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [66]:
### note testen gik meget godt, men det betyder ikke man har en god model ;)
clf.predict(np.array(train_X))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [68]:
test_X = x.toarray()[:int(len(df["type_id"])/2)]
test_y = df["type_id"][:int(len(df["type_id"])/2)]

predictions = clf.predict(np.array(train_X))

failCounter = 0
for i in range(int(len(df["type_id"])/2)):
    if (predictions[i] != test_y[i]):
        failCounter += 1
        
print("Number of mislabeled points out of a total %d points : %d" % (len(test_X), failCounter))

Number of mislabeled points out of a total 125 points : 61
